# Aim
Given a numerplate output the amount of vehicle excise duty ('road tax') payable.

# Motivation
Purchased a 2012 car via autotrader that was advertised as £30 VED by the dealer and by autotrader themselves. Turns out the VED is £100. Autotrader don't guarantee the accuracy of data provided in their T&Cs.

# Background
Revenue from vehicle tax is added to the central funds of the exchequer.

The system for cars registered before 1 April 2017 will not change (according to government). For cars registered on 1 April 2017 onwards there is a simple system of £0 for cars with 0 emissions and a standard rate of £140 for non-zero emission vehicles. There is an additional charge £310 for 5 years for vehicles over £40,000 as well as a special first year rate for new cars that depends on emissions. Source: https://www.gov.uk/government/publications/vehicle-excise-duty/vehicle-excise-duty

We are only interested in vehicles registered before 1 April 2017 due to the simplicity of the other system as mentioned above. Bare in mind the system is subject to change.

# Limitations
For simplicity we will ignore the first year rate on post 2017 cars (which in most cases is close or lower than £140) and the (£310 additional charge for 5 years on cars over £40,0000). These may be added later but are not important for our use case.

# Pseudo Code
1) Accept numberplate from user
2) Submit form with given numberplate here: https://vehicleenquiry.service.gov.uk/?locale=en
3) Ask user if given vehicle is correct.
4) If vehicle is correct submit yes to website. If not try again.
5) Extract CO2 emissions data and date of register from given information.
6) Use the information to determine vehicle tax.
7) Present vehicle tax to user along with any other relevant bonus data.

In [17]:
from bs4 import BeautifulSoup
import requests

#numerplate = input("Enter numberplate: ")
numerplate = "BL61LKE"

url = "https://vehicleenquiry.service.gov.uk/?locale=en"

r = requests.get(url)
soup = BeautifulSoup(r.content, 'lxml')
message = soup.find("noscript")
print(message.prettify())

<noscript>
 Please enable JavaScript to view the page content.
 <br/>
 Your support ID is:   589635766418721698.
</noscript>



# Road bump
The requests module does not execute client-side javascript so the page is unable to render the form required to make the enquiry. I went to the webpage using chrome with javascipt disabled to confirm this.

We will try to submit a post request to the form directly. The form tag looks like this
```html
<form action="/vehicle_enquiry/save?locale=en" accept-charset="UTF-8" method="post">  .... </form>
```
Within the tag it has 2 inputs:
```html
<input type="hidden" name="authenticity_token" value="zwGc....mEWrdA" autocomplete="off" />
<input class="govuk-input govuk-input--width-10" maxlength="8" autocomplete="off" aria-describedby="vrn_field_hint wizard_vehicle_enquiry_capture_vrn_vrn_error" size="8" type="text" name="wizard_vehicle_enquiry_capture_vrn[vrn]" id="wizard_vehicle_enquiry_capture_vrn_vrn" />
```
Note that one input is an `authenticity_token` which is given to you when you have javascript enabled.

In [40]:
import random
import string
from time import sleep

def random_string(length = 25):
    return ''.join(random.choices(string.ascii_lowercase + string.digits + string.ascii_uppercase, k=length))

def random_numberplate():
    return ''.join(random.choices(string.ascii_uppercase + string.digits, k=6))

url = 'https://vehicleenquiry.service.gov.uk/vehicle_enquiry/save?locale=en'
form_data = {'authenticity_token' : random_string(48 + random.randint(0,30)),
        'wizard_vehicle_enquiry_capture_vrn[vrn]' : random_numberplate(),
        }
sleep(random.uniform(0.5,2))
r = requests.post(url, data=form_data)

soup = BeautifulSoup(r.content, 'html.parser')
print(soup.find('noscript').prettify())

<noscript>
 Please enable JavaScript to view the page content.
 <br/>
 Your support ID is:  9645927706275280806.
</noscript>



Again javascript is required to render the response.

The network tab (in browser developer tools) doesn't show any api call or similar that we could send a request to directly. It is client-side javascript that needs executing.

Looks like we have to use selenium /w a headless web browser driver

# Using selenium
Selenium can be used as a python module as well as a Firefox extension. The web extension provides an easy to use GUI that can be converted to code and helps with trying things out. From using Selenium it is clear that the start button changes ID every time you load the page to make it more difficult for bots to locate the button.

We will circumvent the need for the clicking the start button by starting our script at a later url in the chain of events